In [1]:
import numpy as np
import csv
import pandas as pd
from math import sqrt
import regex as re
import matplotlib.pyplot as plt
%matplotlib inline

class Anime:

    def __init__(self,anime_id,name,episodes,public_rating,members):
        self.anime_id = anime_id
        self.name = name
        self.public_rating = public_rating
        self.members = members
        self.episodes = episodes
    
class Rating:
    
    def __init__(self,user_id,anime_id,user_rating):
        self.user_id = user_id
        self.anime_id = anime_id
        self.user_rating = user_rating
        
class Input:
    def __init__(self,input_name,input_rating):
        self.name = input_name 
        self.input_rating = input_rating

In [3]:
def get_anime_info():
    # Reads anime data from a CSV file, cleans it using pandas, and creates a list of Anime objects.
    # Args: csv_file: The path to the CSV file containing anime data.
    # Returns: A list of Anime class objects.
    anime = pd.read_csv('anime.csv')
    anime = anime.dropna()
    anime = anime.drop(columns = ['genre','type'])
    anime['name'] = [re.sub(r'[^a-zA-Z0-9\s]', '', i)for i in anime['name']]
    anime['name'] = anime['name'].str.strip().str.lower()

    # Create Anime objects from the cleaned DataFrame
    animes = anime.apply(lambda row: Anime(*row.tolist()), axis=1).tolist()

    return animes

animes = get_anime_info()
print(animes)

[<__main__.Anime object at 0x000001796EF522D0>, <__main__.Anime object at 0x000001796EBE8190>, <__main__.Anime object at 0x000001796DD61B50>, <__main__.Anime object at 0x000001796EF52310>, <__main__.Anime object at 0x000001796EF52350>, <__main__.Anime object at 0x000001796EF52390>, <__main__.Anime object at 0x000001796EF523D0>, <__main__.Anime object at 0x000001796EF52410>, <__main__.Anime object at 0x000001796EF52450>, <__main__.Anime object at 0x000001796EF52490>, <__main__.Anime object at 0x000001796EF524D0>, <__main__.Anime object at 0x000001796EF52510>, <__main__.Anime object at 0x000001796EF52550>, <__main__.Anime object at 0x000001796EF52590>, <__main__.Anime object at 0x000001796EF525D0>, <__main__.Anime object at 0x000001796EF52610>, <__main__.Anime object at 0x000001796EF52650>, <__main__.Anime object at 0x000001796EF52690>, <__main__.Anime object at 0x000001796EF526D0>, <__main__.Anime object at 0x000001796EE33ED0>, <__main__.Anime object at 0x000001796E5258D0>, <__main__.An

In [12]:
rating = pd.read_csv('rating.csv')
rating = rating.dropna()
rating

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [4]:
def get_rating_info():
    # Reads ratuing data from a CSV file, cleans it using pandas, and creates a list of rating objects.
    # Args: csv_file: The path to the CSV file containing anime rating data.
    # Returns: A list of Rating class objects.
    rating = pd.read_csv('rating.csv')
    rating = rating.dropna()
    
    # Create Anime objects from the cleaned DataFrame
    ratings = rating.apply(lambda row: Rating(*row.tolist()), axis=1).tolist()

    return ratings
ratings = get_rating_info()
print(ratings)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=10000000.0 (bytes/sec)
NotebookApp.rate_limit_window=10.0 (secs)



In [8]:
Steve = [
            {'name':'Gintama', 'rating':9},
            {'name':'Violence Gekiga David no Hoshi', 'rating':7},
            {'name':'Violence Gekiga Shin David no Hoshi Inma Densetsu', 'rating':9},
            {'name':"Yasuji no Pornorama Yacchimae", 'rating':5},
            {'name':'Fullmetal Alchemist Brotherhood', 'rating':9},
            {'name':'Kimi no Na wa', 'rating':6.5},
         ]
steve_data = pd.DataFrame(Steve)
steve_data['name'] = steve_data['name'].str.strip().str.lower()
steve_data

,name,rating
0,gintama,9.0
1,violence gekiga david no hoshi,7.0
2,violence gekiga shin david no hoshi inma densetsu,9.0
3,yasuji no pornorama yacchimae,5.0
4,fullmetal alchemist brotherhood,9.0
5,kimi no na wa,6.5


In [15]:
#getting the User subset data 
def get_user_subset_group(steve_data):
    #getting the anime_data 
    anime = pd.read_csv('anime.csv', usecols=(['anime_id','name']))
    anime = anime.dropna()
    anime['name'] = [re.sub(r'[^a-zA-Z0-9\s]', '', i)for i in anime['name']]
    anime['name'] = anime['name'].str.strip().str.lower()
    
    #getting the rating_data 
    rating = pd.read_csv('rating.csv')
    
    # Filtering out animes by title
    Id = anime[anime['name'].isin(steve_data['name'].tolist())]
    
    # Merging data to get anime ID
    input_data = pd.merge(Id, steve_data, on='name')
    
    # Filtering out users that have watched animes the input user has watched
    users = rating[rating['anime_id'].isin(input_data['anime_id'].tolist())]
    
    # Grouping users by user_id
    userSubsetGroup = users.groupby('user_id')
    
    # Sorting user groups based on the number of common animes watched
    userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]>=5), reverse=True)
    
    # Filter out users who have watched at least 5 similar anime
    userSubsetGroup = [(user_id, group) for user_id, group in userSubsetGroup if len(group) >= 4]
    
    return userSubsetGroup,input_data,anime,rating

# calling the function:
userSubsetGroup,input_data,anime,rating = get_user_subset_group(steve_data)
print(len(userSubsetGroup))
userSubsetGroup

139


[(27364,
           user_id  anime_id  rating
  2944712    27364       918      -1
  2945344    27364      5114       9
  2945401    27364      5621      -1
  2945446    27364      6133      -1
  2946544    27364     26081      -1
  2946735    27364     32281      -1),
 (48766,
           user_id  anime_id  rating
  5097135    48766       918      -1
  5100254    48766      5114      -1
  5100522    48766      5621      -1
  5100757    48766      6133      -1
  5105671    48766     26081      -1
  5106033    48766     28977      -1),
 (1116,
          user_id  anime_id  rating
  108046     1116       918       7
  108076     1116      5114       8
  108350     1116     28977       9
  108388     1116     32281       6),
 (1309,
          user_id  anime_id  rating
  129106     1309       918       9
  129155     1309      5114       8
  129332     1309     28977      10
  129367     1309     32281       9),
 (1522,
          user_id  anime_id  rating
  151332     1522       918       8


In [16]:
#getting simillarity index using the person's coefficient
def get_prearson_coefficient():
    #Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
    pearsonCorDict = {}

    #For every user group in our subset
    for name, group in userSubsetGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='anime_id')
        inputMovie = input_data.sort_values(by='anime_id')
        #Get the N for the formula
        n = len(group)
        #Get the review scores for the movies that they both have in common
        temp = input_data[input_data['anime_id'].isin(group['anime_id'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp['rating'].tolist()
        #put the current user group reviews in a list format
        tempGroupList = group['rating'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)

        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorDict[name] = 0

    pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['user_Id'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    return pearsonDF

pearsonDF = get_prearson_coefficient()
pearsonDF

,similarityIndex,user_Id
0,0.411706,27364
1,0.000000,48766
2,0.258199,1116
3,0.000000,1309
4,0.577350,1522
...,...,...
134,0.000000,71498
135,0.333333,71942
136,0.000000,72007
137,-0.333333,72997


In [18]:
def get_recommended_animes(pearsonDF,rating,anime):
    # Sort users by similarity index
    topUsers = pearsonDF.sort_values(by='similarityIndex', ascending=False)
    
    # Filter top users based on similarity threshold
    topUsers = topUsers[topUsers['similarityIndex'] >= 0.90]
    
    # Initialize list to store related anime IDs
    anime_list = []
    
    # Iterate over top users
    for user_id in topUsers.index:
        # Filter ratings of the current user with high ratings
        related_anime_id = rating[(rating['user_id'] == user_id) & (rating['rating'] >= 9)]['anime_id']
        # Append the related anime IDs to the list
        anime_list.extend(related_anime_id)
    
    # Concatenate the list of related anime IDs into a DataFrame
    new_anime_data = pd.DataFrame({'anime_id': anime_list})
    
    # Drop duplicate anime IDs
    unique_anime_df = pd.DataFrame({'recommended_animes': new_anime_data['anime_id'].drop_duplicates()})
    
    #getting names from the dataframe of actucal animes
    Id = anime[anime['anime_id'].isin(unique_anime_df['recommended_animes'].tolist())]
 
    return Id

anime = get_recommended_animes(pearsonDF,rating,anime)
anime

,anime_id,name
6,11061,hunter x hunter 2011
10,4181,clannad after story
13,2904,code geass hangyaku no lelouch r2
18,12355,ookami kodomo no ame to yuki
19,1575,code geass hangyaku no lelouch
...,...,...
5710,12581,asa made jugyou chu
6086,7739,11eyes momoiro genmutan
11233,3375,kirepapa
12209,5391,pico to chico
